In [1]:
import pandas as pd
import torch
from sklearn.metrics.pairwise import cosine_similarity
import os
from sklearn.metrics.pairwise import cosine_similarity
from InstructorEmbedding import INSTRUCTOR
import numpy as np

model = INSTRUCTOR('hkunlp/instructor-large')

/home/geyu/projects/my-test/.venv/lib/python3.11/site-packages/InstructorEmbedding/instructor.py:7: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import trange


load INSTRUCTOR_Transformer
max_seq_length  512


In [2]:
inference_results = './inference_results.csv'
inference_results_df = pd.read_csv(inference_results)
inference_results_df = inference_results_df.dropna(subset=['transcription']).reset_index(drop=True)

HOT_WORDS = ["be careful", "destroy",  "stranger"]

def building_query(hot_words):
    return [[f"Represent words to find similar phases: ",  f"{', '.join(hot_words)}"]]

def build_corpus(inference_results):
    df = inference_results
    return [['Represent the sentence to find similar phases: ', df['transcription'][i].lower()] for i in range(len(df))] # change to lower to adhere embedding model's example

In [3]:
query = building_query(HOT_WORDS)
corpus = build_corpus(inference_results_df)

# corpus = corpus[:1]
# print(corpus[0])
embeddings_a = model.encode(query)
embeddings_b = model.encode(corpus)
similarities = cosine_similarity(embeddings_a,embeddings_b)

In [4]:
similarities

array([[0.8874334 , 0.78662753, 0.7732773 , ..., 0.78608906, 0.7256171 ,
        0.7441651 ]], shape=(1, 4075), dtype=float32)

In [5]:
sim_arr = np.array(similarities[0])

In [6]:
(sim_arr >= 0.8).sum() # Assuming you want to count how many similarities are above 0.8

np.int64(136)

In [7]:
len(sim_arr) # Count of phrases with similarity above 0.8

4075

In [8]:
inference_results_df['similarity'] = sim_arr >= 0.8
inference_results_df = inference_results_df.drop(columns=['transcription'])
inference_results_df.to_csv('cv-valid-dev-updated.csv', index=False)